In [1]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: cuda


In [3]:
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))

In [4]:
from sam2.build_sam import build_sam2_video_predictor

sam2_checkpoint = "../checkpoints/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"


predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)

In [5]:

# bbox_path = '../../ai_object_bbox_generation/detr/outputs/bboxes/bboxes.json'


# # Load your JSON data
# with open(bbox_path, 'r') as f:
#     data = json.load(f)
#     print(data)

#     # Mapping classes to unique object IDs
# class_to_obj_id = {
#     "bvm": 1,
#     "hands": 2,
#     "defib pads": 3
# }

# img = None
# # Iterate through video files and frames
# for video, frames in data.items():

#     # load video dir
#     video_dir = f'/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/{video}/original/'
        
#     inference_state = predictor.init_state(video_path=video_dir)

#     predictor.reset_state(inference_state)

#     print(f'Processing video: {video}')

#     for frame_info in frames:
#         frame_number = frame_info['frame_counter']
#         bboxes = frame_info['bboxes']

#         print(f'Processing frame: {frame_number}')

#                 # Filter to only get the bbox with highest confidence for class 'bvm'
#         highest_conf_bbox = None
#         for bbox in bboxes:
#             class_name = bbox['class']
#             confidence = bbox['confidence']

#             if class_name == "bvm":
#                 # If no bbox selected yet or if current bbox has higher confidence, select this one
#                 if highest_conf_bbox is None or confidence > highest_conf_bbox['confidence']:
#                     highest_conf_bbox = bbox
        
#         ann_frame_idx = frame_number  # the frame index we interact with

#         print("Highest", highest_conf_bbox)
#         # Draw bounding boxes
#         if highest_conf_bbox:
#             # bbox is a dictionary with format: {'class': 'bvm', 'confidence': 0.9831042289733887, 'xmin': 1717.2135009765625, 'ymin': 517.65234375, 'xmax': 1920.054931640625, 'ymax': 895.0321044921875}
#             xmin, ymin, xmax, ymax = int(highest_conf_bbox['xmin']), int(highest_conf_bbox['ymin']), int(highest_conf_bbox['xmax']), int(highest_conf_bbox['ymax'])
#             confidence = highest_conf_bbox['confidence']


#             ann_obj_id = class_to_obj_id.get(class_name, 4)  # Default to 4 if class not found

#             print(f'Annotation object id: {ann_obj_id}')
#             # Let's add a box at (x_min, y_min, x_max, y_max) = (300, 0, 500, 400) to get started
#             box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
#             _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
#                 inference_state=inference_state,
#                 frame_idx=ann_frame_idx,
#                 obj_id=ann_obj_id,
#                 box=box,
#             )

#             # show the results on the current (interacted) frame
#             plt.figure(figsize=(9, 6))
#             plt.title(f"frame {ann_frame_idx}")
#             plt.imshow(Image.open(os.path.join(video_dir, f"{ann_frame_idx:04d}.jpg")))
#             show_box(box, plt.gca())
#             show_mask((out_mask_logits[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])


#             # break

#         # break
#     break

# Use tracking


In [6]:

def frame_array(video_dir):
    """
    Create an array of frame names formatted as '0000.jpg', based on frame_counter in each frame_info.
    """

    # scan all the JPEG frame names in this directory
    frame_names = [
        p for p in os.listdir(video_dir)
        if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
    ]
    frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))


    return frame_names


def get_bbox_for_video(bbox_json_path):
    """
    Get the first frame that has the highest confidence bounding box for each object class in the video.
    
    Parameters:
    - bbox_json_path: Path to the JSON file containing bounding box data.
    
    Returns:
    A dictionary containing the video directory, bounding box coordinates, confidence, 
    annotation object ID, annotation frame index, object class, and frame names for each class.
    """
    # Load JSON data
    with open(bbox_json_path, 'r') as f:
        data = json.load(f)

    # Mapping object classes to unique IDs
    class_to_obj_id = {
        "bvm": 1,
        "hands": 2,
        "defib pads": 3
    }

    video_data = {}

    # Iterate over each video and its frames
    for video, frames in data.items():
        video_dir = f'/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/{video}/original/'
        print(f'Processing video: {video}')

        # Generate frame names
        frame_names = frame_array(video_dir)

        # Dictionary to store the first frame with the highest confidence bbox for each class
        highest_conf_bboxes = {}

        # Iterate over frames to find the first highest confidence bbox for each object class
        for frame_info in frames:
            frame_number = frame_info['frame_counter']
            bboxes = frame_info['bboxes']
            actual_frame = frame_info['frame']

            # Check each bounding box for its class and confidence
            for bbox in bboxes:
                class_name = bbox['class']
                confidence = bbox['confidence']

                # If the class hasn't been processed yet, add it to the results
                if class_name not in highest_conf_bboxes:
                    highest_conf_bboxes[class_name] = {
                        'bbox': [int(bbox['xmin']), int(bbox['ymin']), int(bbox['xmax']), int(bbox['ymax'])],
                        'confidence': confidence,
                        'ann_obj_id': class_to_obj_id.get(class_name, 4),
                        'ann_frame_idx': frame_number,
                        'actual_frame': actual_frame
                    }

        # If we found bounding boxes for any classes, store the video data
        if highest_conf_bboxes:
            video_data[video] = {
                'video_dir': video_dir,
                'first_conf_bboxes': highest_conf_bboxes,
                'frame_names': frame_names
            }

    return video_data if video_data else None

# Specify the path to the JSON file containing bounding box data
bbox_json_path = '../../ai_object_bbox_generation/detr/outputs/bboxes/bboxes.json'

# Get the bounding box for the specified object class
result = get_bbox_for_video(bbox_json_path)

# Print the result
if result:
    # print dict pretty
    print(f"All video bbox data")
    print(json.dumps(result, indent=4))

Processing video: GX010312_clipped_with_audio
Processing video: GX010309_clipped_with_audio
Processing video: GX010308_clipped_with_audio
Processing video: GX010305_clipped_with_audio
Processing video: GX010306_clipped_with_audio
Processing video: GX010307_clipped_with_audio
Processing video: GX010310_clipped_with_audio
Processing video: GX010311_clipped_with_audio
Processing video: GX010314_clipped_with_audio
Processing video: GX010313_clipped_with_audio
All video bbox data
{
    "GX010312_clipped_with_audio": {
        "video_dir": "/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/GX010312_clipped_with_audio/original/",
        "first_conf_bboxes": {
            "bvm": {
                "bbox": [
                    1717,
                    517,
                    1920,
                    895
                ],
                "confidence": 0.9831042289733887,
                "ann_obj_id": 1,
                "ann_frame_idx": 0,
     

## SAM 2 Video Tracking and Segmentation Mask Generation

In [7]:
# Helper functions

def add_objects_to_predictor(predictor,inference_state, bbox, ann_obj_id, ann_frame_idx):
    """
    Track the object and generate the mask for the object in the video.
    
    Parameters:
    - bbox: The bounding box coordinates for the object.
    - video_dir: The directory containing the video frames.
    - predictor: The SAM2 video predictor model.
    - ann_obj_id: The annotation object ID for the object.
    - ann_frame_idx: The annotation frame index for the object.
    
    Returns:
    A dictionary containing the per-frame segmentation results for the video.
    """

    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=ann_obj_id,
        box=bbox,
    )

    # Return the predictor state
    return _, out_obj_ids, out_mask_logits 

def track_and_generate_mask(predictor, inference_state):
        # run propagation throughout the video and collect the results in a dict
    video_segments = {}  # video_segments contains the per-frame segmentation results
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    return video_segments

def render_video_segments(ann_frame_idx, frame_names, video_dir, video_segments, vis_frame_stride=10):
        # render the segmentation results every few frames
    plt.close("all")
    for out_frame_idx in range(ann_frame_idx, len(frame_names), vis_frame_stride):
        plt.figure(figsize=(6, 4))
        plt.title(f"frame {out_frame_idx}")
        plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            show_mask(out_mask, plt.gca(), obj_id=out_obj_id)


def render_video_segments_to_video(ann_frame_idx, frame_names, video_dir, video_segments, output_video_path, fps=30):
    """
    Renders the segmentation results on video frames and saves them as an MP4 video.

    Parameters:
    - ann_frame_idx: The starting frame index from which to render the video.
    - frame_names: List of frame file names.
    - video_dir: Directory where the video frames are located.
    - video_segments: Segmentation results for each frame.
    - output_video_path: Path to save the output video.
    - fps: Frames per second for the output video.
    """


    # Get the dimensions of the first frame
    # first_frame_path = os.path.join(video_dir, frame_names[ann_frame_idx])
    # first_frame = Image.open(first_frame_path)
    # frame_width, frame_height = first_frame.size

    # Define the codec and create VideoWriter object to save MP4
    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    # video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    for out_frame_idx in range(ann_frame_idx, len(frame_names)):
        # print(f"Rendering frame {out_frame_idx}...")
        # Plot the image with segmentation results
        plt.figure(figsize=(6, 4))
        plt.title(f"Frame {out_frame_idx}")
        frame_path = os.path.join(video_dir, frame_names[out_frame_idx])
        # Load and display the image, but don't show the plot
        frame = Image.open(frame_path)
        plt.imshow(frame)

        # Draw segmentation masks
        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            show_mask(out_mask, plt.gca(), obj_id=out_obj_id)
        
        # Save the current figure as an image in memory
        plt.axis('off')  # Remove axes for cleaner output
        plt.tight_layout()
        plt.savefig(f"{output_video_path}/segmented_{out_frame_idx}.png", bbox_inches='tight', pad_inches=0)
        plt.close()

        # # Load the saved image and convert to the correct format for video
        # temp_frame = cv2.imread("temp_frame.png")
        # # temp_frame = cv2.cvtColor(temp_frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # # Resize the frame if necessary to match the original video frame size
        # if temp_frame.shape[1] != frame_width or temp_frame.shape[0] != frame_height:
        #     temp_frame = cv2.resize(temp_frame, (frame_width, frame_height))

        # # Write the frame to the video
        # video_writer.write(temp_frame)

    # Release the VideoWriter object
    # video_writer.release()
    # print(f"Video saved to {output_video_path}")

    # Clean up temporary files
    # if os.path.exists("temp_frame.png"):
    #     os.remove("temp_frame.png")

: 

In [8]:

# Specify the object class for which to extract the bounding box
result = get_bbox_for_video(bbox_json_path)
# Print the result
if result:
    # print dict pretty
    print(f"All video bbox data")
    print(json.dumps(result, indent=4))



# Iterate over video data and process the frames

for video, data in result.items():
    video_dir = data['video_dir']
    frame_names = data['frame_names']

    print(len(frame_names))

    print(f'Processing video: {video}')

    # # initialize the video predictor
    inference_state = predictor.init_state(video_path=video_dir)
    predictor.reset_state(inference_state)

    for class_name, class_data in data['first_conf_bboxes'].items():

        print(f'Processing class: {class_name}')
        print(f'Class data: {class_data}')

        bbox = class_data['bbox']
        object_class = class_name
        confidence = class_data['confidence']
        ann_obj_id = class_data['ann_obj_id']
        # ann_frame_idx = class_data['ann_frame_idx'] # this is if original frames are also downsampled to bbox generation frame rate
        ann_frame_idx = class_data['actual_frame']

        print(f'Bbox: {bbox}, ann_obj_id: {ann_obj_id}, ann_frame_idx: {ann_frame_idx}')

        if object_class == 'hands':
            print('Skipping hands')
            continue
        # Add the object to the predictor
        add_objects_to_predictor(predictor, inference_state, bbox, ann_obj_id, ann_frame_idx)


    # Track the object and generate the mask for the object in the video
    video_segments = track_and_generate_mask(predictor, inference_state)

    # # Display the segmentation results for the video
    # vis_frame_stride=30
    # render_video_segments(ann_frame_idx, frame_names, video_dir, video_segments,vis_frame_stride)

    # # Save the segmentation results as an MP4 video
    output_video_path = f"{video_dir}/segmentations/"
    os.makedirs(output_video_path, exist_ok=True)
    # output_video_path = f"{output_video_path}/{video}_segmentation.mp4"

    render_video_segments_to_video(0, frame_names, video_dir, video_segments, output_video_path, fps=30)

    break

    

Processing video: GX010312_clipped_with_audio
Processing video: GX010309_clipped_with_audio
Processing video: GX010308_clipped_with_audio
Processing video: GX010305_clipped_with_audio
Processing video: GX010306_clipped_with_audio
Processing video: GX010307_clipped_with_audio
Processing video: GX010310_clipped_with_audio
Processing video: GX010311_clipped_with_audio
Processing video: GX010314_clipped_with_audio
Processing video: GX010313_clipped_with_audio
All video bbox data
{
    "GX010312_clipped_with_audio": {
        "video_dir": "/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/GX010312_clipped_with_audio/original/",
        "first_conf_bboxes": {
            "bvm": {
                "bbox": [
                    1717,
                    517,
                    1920,
                    895
                ],
                "confidence": 0.9831042289733887,
                "ann_obj_id": 1,
                "ann_frame_idx": 0,
     

frame loading (JPEG): 100%|██████████| 2831/2831 [02:22<00:00, 19.91it/s]


Processing class: bvm
Class data: {'bbox': [1717, 517, 1920, 895], 'confidence': 0.9831042289733887, 'ann_obj_id': 1, 'ann_frame_idx': 0, 'actual_frame': 0}
Bbox: [1717, 517, 1920, 895], ann_obj_id: 1, ann_frame_idx: 0
Processing class: hands
Class data: {'bbox': [769, 989, 1031, 1079], 'confidence': 0.9920879006385803, 'ann_obj_id': 2, 'ann_frame_idx': 0, 'actual_frame': 0}
Bbox: [769, 989, 1031, 1079], ann_obj_id: 2, ann_frame_idx: 0
Skipping hands
Processing class: defib pads
Class data: {'bbox': [1399, 590, 1543, 750], 'confidence': 0.9149645566940308, 'ann_obj_id': 3, 'ann_frame_idx': 2, 'actual_frame': 58}
Bbox: [1399, 590, 1543, 750], ann_obj_id: 3, ann_frame_idx: 58


/sfs/weka/scratch/cjh9fw/Rivanna/2024/repos/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/sfs/weka/scratch/cjh9fw/Rivanna/2024/repos/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 2831/2831 [02:32<00:00, 18.56it/s]


In [15]:
# render_video_segments_to_video(0, frame_names, video_dir, video_segments, output_video_path, fps=30)


: 

In [ ]:

bbox_path = '../../ai_object_bbox_generation/detr/outputs/bboxes/bboxes.json'


# Load your JSON data
with open(bbox_path, 'r') as f:
    data = json.load(f)
    print(data)

    # Mapping classes to unique object IDs
class_to_obj_id = {
    "bvm": 1,
    "hands": 2,
    "defib pads": 3
}

img = None
highest_conf_bbox = None    
# Iterate through video files and frames

# create frame_names 
frame_names = []

for video, frames in data.items():

    # load video dir
    video_dir = f'/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/{video}/original/'
        
    print(f'Processing video: {video}')

    for frame_info in frames:
        frame_number = frame_info['frame_counter']
        frame_names.append(f"{frame_number:04d}.jpg")


    # load video dir
    video_dir = f'/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/{video}/original/'
    print(f'Processing video: {video}')

    for frame_info in frames:
        frame_number = frame_info['frame_counter']
        bboxes = frame_info['bboxes']

        print(f'Processing frame: {frame_number}')
                # Filter to only get the bbox with highest confidence for class 'bvm'
        highest_conf_bbox = None
        for bbox in bboxes:
            class_name = bbox['class']
            confidence = bbox['confidence']

            if class_name == "bvm":
                # If no bbox selected yet or if current bbox has higher confidence, select this one
                if highest_conf_bbox is None or confidence > highest_conf_bbox['confidence']:
                    highest_conf_bbox = bbox
        
        ann_frame_idx = frame_number  # the frame index we interact with

        print("Highest", highest_conf_bbox)
        if highest_conf_bbox is not None:
            # Draw bounding boxes
                
            inference_state = predictor.init_state(video_path=video_dir)

            predictor.reset_state(inference_state)

            # bbox is a dictionary with format: {'class': 'bvm', 'confidence': 0.9831042289733887, 'xmin': 1717.2135009765625, 'ymin': 517.65234375, 'xmax': 1920.054931640625, 'ymax': 895.0321044921875}
            xmin, ymin, xmax, ymax = int(highest_conf_bbox['xmin']), int(highest_conf_bbox['ymin']), int(highest_conf_bbox['xmax']), int(highest_conf_bbox['ymax'])
            confidence = highest_conf_bbox['confidence']
            ann_obj_id = class_to_obj_id.get(class_name, 4)  # Default to 4 if class not found

            box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
            _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=ann_frame_idx,
                obj_id=ann_obj_id,
                box=box,
            )
            # run propagation throughout the video and collect the results in a dict
            video_segments = {}  # video_segments contains the per-frame segmentation results
            for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
                video_segments[out_frame_idx] = {
                    out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                    for i, out_obj_id in enumerate(out_obj_ids)
                }


            # render the segmentation results every few frames
            vis_frame_stride = 10
            plt.close("all")
            for out_frame_idx in range(ann_frame_idx, len(frame_names), vis_frame_stride):
                plt.figure(figsize=(6, 4))
                plt.title(f"frame {out_frame_idx}")
                plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
                for out_obj_id, out_mask in video_segments[out_frame_idx].items():
                    show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

            break
    break

In [ ]:

highest_conf_bbox = None    
# Iterate through video files and frames
for video, frames in data.items():

    # load video dir
    video_dir = f'/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Tools/ai_object_bbox_generation/detr/outputs/bboxes/{video}/original/'
    print(f'Processing video: {video}')

    for frame_info in frames:
        frame_number = frame_info['frame_counter']
        bboxes = frame_info['bboxes']

        print(f'Processing frame: {frame_number}')
                # Filter to only get the bbox with highest confidence for class 'bvm'
        highest_conf_bbox = None
        for bbox in bboxes:
            class_name = bbox['class']
            confidence = bbox['confidence']

            if class_name == "defib pads":
                # If no bbox selected yet or if current bbox has higher confidence, select this one
                if highest_conf_bbox is None or confidence > highest_conf_bbox['confidence']:
                    highest_conf_bbox = bbox
        
        ann_frame_idx = frame_number  # the frame index we interact with

        print("Highest", highest_conf_bbox)
        if highest_conf_bbox is not None and ann_frame_idx%30 == 0:


            # Draw bounding boxes
                
            inference_state = predictor.init_state(video_path=video_dir)

            predictor.reset_state(inference_state)

            # bbox is a dictionary with format: {'class': 'bvm', 'confidence': 0.9831042289733887, 'xmin': 1717.2135009765625, 'ymin': 517.65234375, 'xmax': 1920.054931640625, 'ymax': 895.0321044921875}
            xmin, ymin, xmax, ymax = int(highest_conf_bbox['xmin']), int(highest_conf_bbox['ymin']), int(highest_conf_bbox['xmax']), int(highest_conf_bbox['ymax'])
            confidence = highest_conf_bbox['confidence']
            ann_obj_id = class_to_obj_id.get(class_name, 4)  # Default to 4 if class not found

            box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
            _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=ann_frame_idx,
                obj_id=ann_obj_id,
                box=box,
            )
            # run propagation throughout the video and collect the results in a dict
            video_segments = {}  # video_segments contains the per-frame segmentation results
            for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
                video_segments[out_frame_idx] = {
                    out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                    for i, out_obj_id in enumerate(out_obj_ids)
                }


            # render the segmentation results every few frames
            vis_frame_stride = 10
            plt.close("all")
            for out_frame_idx in range(ann_frame_idx, len(frame_names), vis_frame_stride):
                plt.figure(figsize=(6, 4))
                plt.title(f"frame {out_frame_idx}")
                plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
                for out_obj_id, out_mask in video_segments[out_frame_idx].items():
                    show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

            # break
    break

In [ ]:
#  load the video

# `video_dir` a directory of JPEG frames with filenames like `<frame_index>.jpg`
video_dir = "./videos/bedroom"
video_dir = "./videos/defib"

# scan all the JPEG frame names in this directory
frame_names = [
    p for p in os.listdir(video_dir)
    if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
]
frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))

# take a look the first video frame
frame_idx = 0
plt.figure(figsize=(9, 6))
plt.title(f"frame {frame_idx}")
plt.imshow(Image.open(os.path.join(video_dir, frame_names[frame_idx])))

In [6]:

bbox_path = '../../ai_object_bbox_generation/detr/outputs/bboxes/bboxes.json'


# Load your JSON data
with open(bbox_path, 'r') as f:
    data = json.load(f)
    print(data)

img = None
# Iterate through video files and frames
for video, frames in data.items():

    # load video file
    video_path = f'//standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/May_2024/DO_NOT_DELETE/ego_camera/clipped_with_audio/{video}.mp4'
        
    inference_state = predictor.init_state(video_path=video_dir)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f'Error opening video file {video_path}')
        continue

    for frame_info in frames:
        frame_number = frame_info['frame']
        bboxes = frame_info['bboxes']

        print(video, frame_number, bboxes)

        # Read frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if not ret:
            print(f'Error reading frame {frame_number} from video {video}')
            continue

        # Draw bounding boxes
        for bbox in bboxes:
            # bbox is a dictionary with format: {'class': 'bvm', 'confidence': 0.9831042289733887, 'xmin': 1717.2135009765625, 'ymin': 517.65234375, 'xmax': 1920.054931640625, 'ymax': 895.0321044921875}
            xmin, ymin, xmax, ymax = int(bbox['xmin']), int(bbox['ymin']), int(bbox['xmax']), int(bbox['ymax'])
            class_name = bbox['class']
            confidence = bbox['confidence']
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            img = frame

            if(confidence > 0.9):
                # break the entire loop
                break

        break
        # Display frame
        # cv2.imshow('Frame', frame)
        # cv2.waitKey(0)

        print(video, frame_number, bboxes)

In [7]:
# show image in jupyer notebook
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)